# Virtual Piano Tutor

# Setup

## Libraries

In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

from vpt.features.features import *
import vpt.utils.image_processing as ip
import vpt.settings as s
import vpt.hand_detection.depth_context_features as dcf

%load_ext autoreload
%autoreload 2

## Some helper functions

### Load/Save Data Set

In [2]:
def load_data(testing_p, M, radius, feature_type, data_type):
    base = "data/posture/extracted/"
    data_path = os.path.join(base, "{}_M{}_rad{:0.2f}_{}_".format(testing_p, M, radius, feature_type))
    data = np.load(data_path + data_type + "_data_combined.npz")    
    return data

## Project Setup

### Generate or Load Data

In [3]:
M = 5
radius = .15
feature_type = "exp1-hog"
testing_p = "p3"

In [4]:
#### Load data for a single paricipant
data = load_data("all_participants", M, radius, feature_type, "train")
print("X LH", data["X_lh"].shape, "y LH", data["y_lh"].shape)
print("X RH", data["X_rh"].shape, "y RH", data["y_rh"].shape)
print("Filenames", data["filenames"].shape)

X LH (15817, 15876) y LH (15817,)
X RH (15817, 15876) y RH (15817,)
Filenames (15817,)


In [5]:
## using p6 for validation
r = re.compile(testing_p)
vmatch = np.vectorize(lambda x:bool(r.search(x)))
val_p = vmatch(data['filenames'])

X_lh = data['X_lh'][~val_p]
y_lh = data['y_lh'][~val_p]
X_rh = data['X_rh'][~val_p]
y_rh = data['y_rh'][~val_p]
filenames = data['filenames'][~val_p]


X_lh_test = data['X_lh'][val_p]
y_lh_test = data['y_lh'][val_p]
X_rh_test = data['X_rh'][val_p]
y_rh_test = data['y_rh'][val_p]
filenames_test = data['filenames'][val_p]

print("Cross Val Data:")
print(X_lh.shape, y_lh.shape)
print(X_rh.shape, y_rh.shape)
print(filenames.shape)
print()
print("Validation Data")
print(X_lh_test.shape, y_lh_test.shape)
print(X_rh_test.shape, y_rh_test.shape)
print(filenames_test.shape)

Cross Val Data:
(11245, 15876) (11245,)
(11245, 15876) (11245,)
(11245,)

Validation Data
(4572, 15876) (4572,)
(4572, 15876) (4572,)
(4572,)


In [6]:
groups = np.zeros_like(filenames, dtype=int)

for p in ["p1", "p3", "p4", "p6"]:
    p_num = int(p[1])
    groups[np.where(np.char.find(filenames, p) != -1)] = p_num
    
print(groups.shape)
print(np.unique(groups))
print(groups)

(11245,)
[1 4 6]
[1 1 1 ..., 6 6 6]


# Classification

### Libraries

In [7]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, LeaveOneGroupOut, GroupKFold
from imblearn.pipeline import Pipeline

from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

## Load Data for Classification

In [8]:
## find all "static" data so we can ignore for now
r = re.compile('p[\d]s')

# remove p#s data
vmatch = np.vectorize(lambda x:bool(r.search(x)))
rem_static = vmatch(filenames)
rem_static_test = vmatch(filenames_test)

X_lh_train, y_lh_train, filenames_train, groups_train = X_lh[~rem_static], y_lh[~rem_static], filenames[~rem_static], groups[~rem_static]
X_rh_train, y_rh_train, filenames_train, groups_train = X_rh[~rem_static], y_rh[~rem_static], filenames[~rem_static], groups[~rem_static]

In [9]:
X_lh_val, y_lh_val, filenames_val = X_lh_test[~rem_static_test], y_lh_test[~rem_static_test], filenames_test[~rem_static_test]
X_rh_val, y_rh_val, filenames_val = X_rh_test[~rem_static_test], y_rh_test[~rem_static_test], filenames_test[~rem_static_test]

In [10]:
X_comb_train = np.vstack((X_lh_train, X_rh_train))
y_comb_train = np.hstack((y_lh_train, y_rh_train))
groups_comb_train = np.hstack((groups_train, groups_train))
filenames_comb_train = np.hstack((filenames_train, filenames_train))

X_comb_val = np.vstack((X_lh_val, X_rh_val))
y_comb_val = np.hstack((y_lh_val, y_rh_val))
filenames_comb_val = np.hstack((filenames_val, filenames_val))

In [11]:
print("Cross Validation Data")
print("LH:", X_lh_train.shape, y_lh_train.shape)
print(np.unique(y_lh_train, return_counts=True))
print("RH:", X_rh_train.shape, y_rh_train.shape)
print(filenames_train.shape)
print(groups_train.shape)
print(np.unique(y_rh_train, return_counts=True))
print("Combined:", X_comb_train.shape, y_comb_train.shape)
print(filenames_comb_train.shape)
print(groups_comb_train.shape)
print(np.unique(y_comb_train, return_counts=True))
print()
print()
print("Validation Data")
print("LH:", X_lh_val.shape, y_lh_val.shape)
print(np.unique(y_lh_val, return_counts=True))
print("RH:", X_rh_val.shape, y_rh_val.shape)
print(np.unique(y_rh_val, return_counts=True))
print("Combined:", X_comb_val.shape, y_comb_val.shape)
print(np.unique(y_comb_val, return_counts=True))
print(filenames_comb_val.shape)

Cross Validation Data
LH: (9628, 15876) (9628,)
(array([0, 1, 2]), array([6889, 1718, 1021]))
RH: (9628, 15876) (9628,)
(9628,)
(9628,)
(array([0, 1, 2]), array([7495, 2106,   27]))
Combined: (19256, 15876) (19256,)
(19256,)
(19256,)
(array([0, 1, 2]), array([14384,  3824,  1048]))


Validation Data
LH: (2650, 15876) (2650,)
(array([0, 1, 2]), array([1267, 1336,   47]))
RH: (2650, 15876) (2650,)
(array([0]), array([2650]))
Combined: (5300, 15876) (5300,)
(array([0, 1, 2]), array([3917, 1336,   47]))
(5300,)


In [12]:
y_comb_train_bin = np.zeros_like(y_comb_train)
y_comb_train_bin[y_comb_train != 0] = 1

y_comb_val_bin = np.zeros_like(y_comb_val)
y_comb_val_bin[y_comb_val != 0] = 1

print(np.unique(y_comb_train_bin, return_counts=True))
print(np.unique(y_comb_val_bin, return_counts=True))

(array([0, 1]), array([14384,  4872]))
(array([0, 1]), array([3917, 1383]))


## Model Testing

### SVM

In [13]:
size = 30
groups_kfold = np.zeros_like(filenames_comb_train, dtype=int)
for i in range(len(filenames_comb_train) // size):
    groups_kfold[i * size:i * size + size] = i
groups_kfold[len(groups_kfold) % size * -1:] = i + 1

cv = GroupKFold(n_splits=3)

In [ ]:
## Parameters for SVMs
steps = [('SMOTE', SMOTEENN()), ("SVC", SVC())]
# param_grid = [
# #   {'SVC__C': [1, 10, 100], 'SVC__kernel': ['linear'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
# #   {'SVC__C': [1, 10, 100], 'SVC__gamma': [.0001, .001, .01, .1], 'SVC__kernel': ['rbf'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
#   {'SVC__C': [10, 100], 'SVC__gamma': [.000005, .00001, .00005,], 'SVC__kernel': ['rbf'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2']},
#  ]

steps = [('SMOTE', SMOTEENN()), ("SVC", SVC())]
param_grid = [
  {'SVC__C': [1, 10, 100], 
   'SVC__kernel': ['linear'], 
   'SMOTE__smote': [SMOTE(kind='borderline2')], 
   "SMOTE__enn": [EditedNearestNeighbours(n_neighbors=7, kind_sel='mode'), EditedNearestNeighbours(n_neighbors=5, kind_sel='mode')]},
  {'SVC__C': [1, 10, 100], 
   'SVC__gamma': [.001, .01, .1], 
   'SVC__kernel': ['rbf'], 
   'SMOTE__smote': [SMOTE(kind='borderline2')], 
   "SMOTE__enn": [EditedNearestNeighbours(n_neighbors=7, kind_sel='mode'), EditedNearestNeighbours(n_neighbors=5, kind_sel='mode')]},
 ]


pipeline = Pipeline(steps)

scores = ['accuracy']
logo = GroupKFold()

In [ ]:
# Hyper Parameter Tuning
for score in scores:
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        print("## Tuning hyper-parameters for {}".format(score))
        print()

        if score is "accuracy":
            scoring = score
        else:
            scoring = '{}_weighted'.format(score)      
        
        #### TRAIN COMBINED LH & RH
        clf_comb = GridSearchCV(pipeline, param_grid, cv=logo.split(X_comb_train, y_comb_train_bin,groups=groups_kfold), scoring=scoring, n_jobs=2, verbose=10)
        clf_comb.fit(X_comb_train, y_comb_train_bin)

        print("Best Combined Parameters set found on data set:")
        print()
        print(clf_comb.best_params_)
        print()
        print("Grid scores on data set:")
        print()
        means = clf_comb.cv_results_['mean_test_score']
        stds  = clf_comb.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_comb.cv_results_['params']):
            print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
        print()

In [ ]:
from sklearn.metrics import accuracy_score
y_comb_true, y_comb_pred = y_comb_val_bin, clf_comb.predict(X_comb_val)

print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))
print(classification_report(y_comb_true, y_comb_pred))
print()


### Single Classifier

In [ ]:
# steps = [('PCA', PCA(n_components=1500)), ('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=100, gamma=.0001, kernel='rbf', probability=False))]
steps = [('SMOTE', SMOTE(kind="regular")), ("SVC", SVC(C=100, gamma=.00005, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
pipeline.fit(X_comb_train, y_comb_train)

In [ ]:
y_comb_true, y_comb_pred = y_comb_val, pipeline.predict(X_comb_val)
print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))
print(classification_report(y_comb_true, y_comb_pred))

In [ ]:
window_size = 5
y_comb_true_maj = []
for i in range(0, y_comb_true.size, window_size):
    u, counts = np.unique(y_comb_true[i:i+window_size], return_counts=True)
    pred = u[np.argmax(counts)]
    y_comb_true_maj.append(pred)
    
y_comb_true_maj = np.array(y_comb_true_maj)

In [ ]:
y_comb_pred_maj = []
for i in range(0, y_comb_pred.size, window_size):
    u, counts = np.unique(y_comb_pred[i:i+window_size], return_counts=True)
    pred = u[np.argmax(counts)]
    y_comb_pred_maj.append(pred)
    
y_comb_pred_maj = np.array(y_comb_pred_maj)

In [ ]:
print("rH Validatation Score:", accuracy_score(y_comb_true_maj, y_comb_pred_maj))
print("rH Confustion Matrix:\n", confusion_matrix(y_comb_true_maj, y_comb_pred_maj))
print(classification_report(y_comb_true_maj, y_comb_pred_maj))

### Cross Validation

In [14]:
from sklearn.model_selection import cross_val_score, cross_validate

In [15]:
X_comb_all_cv = np.vstack((X_comb_train, X_comb_val))
y_comb_all_cv = np.hstack((y_comb_train, y_comb_val))
filenames_comb_all_cv = np.hstack((filenames_comb_train, filenames_comb_val))

print("Combined:", X_comb_all_cv.shape, y_comb_all_cv.shape)
print(np.unique(y_comb_all_cv, return_counts=True))
print(filenames_comb_all_cv.shape)

Combined: (24556, 15876) (24556,)
(array([0, 1, 2]), array([18301,  5160,  1095]))
(24556,)


In [22]:
size = 30
groups_kfold = np.zeros_like(filenames_comb_all_cv, dtype=int)
for i in range(len(filenames_comb_all_cv) // size):
    groups_kfold[i * size:i * size + size] = i
groups_kfold[len(groups_kfold) % size * -1:] = i + 1

cv = GroupKFold(n_splits=3)

In [23]:
# groups_cv = np.zeros_like(filenames_comb_all_cv, dtype=int)

# for p in ["p1", "p3", "p4", "p6"]:
#     p_num = int(p[1])
#     groups_cv[np.where(np.char.find(filenames_comb_all_cv, p) != -1)] = p_num
    
print(groups_kfold.shape)
print(np.unique(groups_kfold))
print(groups_kfold)

(24556,)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 2

In [ ]:
# steps = [('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=1, gamma=.001, kernel='rbf', probability=False))]
steps = [("SVC", SVC(C=1, gamma=.001, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
scoring = ("f1_macro", "accuracy")
# logo = LeaveOneGroupOut()
logo = GroupKFold(n_splits=3)
scores = cross_validate(pipeline, X_comb_all_cv, y_comb_all_cv, cv=logo.split(X_comb_all_cv, y_comb_all_cv,groups=groups_kfold), scoring=scoring, n_jobs=2, verbose=2)

[CV]  ................................................................
[CV]  ................................................................


In [ ]:
print(scores)

In [ ]:
for k, v in scores.items():
    print(k, v)

In [ ]:
steps = [('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=1, gamma=.001, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
scoring = ("f1_macro", "accuracy")
logo = LeaveOneGroupOut()
scores = cross_validate(pipeline, X_comb_all_cv, y_comb_all_cv, cv=logo.split(X_comb_all_cv, y_comb_all_cv, groups=groups_cv), scoring=scoring, n_jobs=2, verbose=3)

In [ ]:
for k, v in scores.items():
    print(k, v)

In [ ]:
logo.split(X_comb_all_cv, y_comb_all_cv, groups=groups_cv)